<a href="https://colab.research.google.com/github/jhno-code/2020-2/blob/main/FirstPrivate%20-%20openMP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



---


**FIRSTPRIVATE**


---


Primer código <br>
indx como variable **firstprivate**

In [ ]:
code="""
#include <omp.h>
#include <stdlib.h>
#include <stdio.h>


int main(int argc, char* argv[])
{
  int TID, i;
  int n = 2;                            // datos a editar por hilo
  int indx = 4;                         // índice de inicio
  int vlen = 12;              // longitud del arreglo

  int a[vlen];
  for(i=0; i < vlen; i++) a[i] = -i-1;  // a = [-1, -2, -3, ...]

  #pragma omp parallel firstprivate(indx) private(i,TID) shared(n,a)
  {
      TID = omp_get_thread_num();       // obtener numero de hilo
      indx = indx + n*TID;              // indice de inicio para cada hilo

      for(i=indx; i<indx+n; i++)        // Cambiar valores de "a"
          a[i] = TID;
  } /*-- End of parallel region --*/

  printf("Despues de la región paralela:\\n");
  for (i=0; i<vlen; i++)
    printf("a[%d] = %d\\n",i,a[i]);     // imprimir elementos de "a"
  return 0;
}
"""

text_file = open("codeB.c", "w")
text_file.write(code)
text_file.close()
%env OMP_NUM_THREADS=4
!gcc -o Ejecutar -fopenmp codeB.c
!./Ejecutar

env: OMP_NUM_THREADS=4
Despues de la región paralela:
a[0] = -1
a[1] = -2
a[2] = -3
a[3] = -4
a[4] = 0
a[5] = 0
a[6] = 1
a[7] = 1
a[8] = 2
a[9] = 2
a[10] = 3
a[11] = 3


Segundo código
indx como **private**

In [ ]:
code="""
#include <omp.h>
#include <stdlib.h>
#include <stdio.h>


int main(int argc, char* argv[])
{
  int TID, i, vlen, indx, n = 2, offset = 4;
  vlen = offset + n * 4;

  int a[vlen];
  for(int i=0; i < vlen; i++) a[i] = -i-1;  // a = [-1, -2, -3, ...]

  #pragma omp parallel private(i,TID,indx) shared(n,offset,a)
  {
    TID = omp_get_thread_num();
    indx = offset + n*TID;

    for(i=indx; i<indx+n; i++)
        a[i] = TID;
    } /*-- End of parallel region --*/

  printf("Despues de la región paralela:\\n");
  for (i=0; i<vlen; i++)
    printf("a[%d] = %d\\n",i,a[i]);     // imprimir elementos de "a"
  return 0;
}
"""

text_file = open("codeB.c", "w")
text_file.write(code)
text_file.close()
%env OMP_NUM_THREADS=4
!gcc -o Ejecutar -fopenmp codeB.c
!./Ejecutar

env: OMP_NUM_THREADS=4
Despues de la región paralela:
a[0] = -1
a[1] = -2
a[2] = -3
a[3] = -4
a[4] = 0
a[5] = 0
a[6] = 1
a[7] = 1
a[8] = 2
a[9] = 2
a[10] = 3
a[11] = 3


**firstprivate** y **shared** son parecidos, la diferencia radica en que las variables de **shared** se comparten por todos los hilos a diferencia de **firstprivate** donde cada hilo hace una copia de la variable.



---



In [10]:
code="""
#include <omp.h>
#include <stdlib.h>
#include <stdio.h>


int main(int argc, char* argv[])
{
  int x = 0, y = 0, z = 0;

  #pragma omp parallel private(x) firstprivate(z)
  {
    printf("x: %i, y: %i, z: %i\\n", x,y,z);
    x = 1;
    y = 1;
    z = 1;
    printf("x: %i, y: %i, z: %i\\n", x,y,z);
  } /*-- End of parallel region --*/

  printf("x: %i, y: %i, z: %i\\n", x,y,z);
  return 0;
}
"""

text_file = open("codeB.c", "w")
text_file.write(code)
text_file.close()
%env OMP_NUM_THREADS=4
!gcc -o Ejecutar -fopenmp codeB.c
!./Ejecutar

env: OMP_NUM_THREADS=4
x: 0, y: 0, z: 0
x: 1, y: 1, z: 1
x: 0, y: 0, z: 0
x: 1, y: 1, z: 1
x: 0, y: 1, z: 0
x: 1, y: 1, z: 1
x: 0, y: 1, z: 0
x: 1, y: 1, z: 1
x: 0, y: 1, z: 0




---

**LastPrivate**

In [42]:
code="""
#include <omp.h>
#include <stdlib.h>
#include <stdio.h>


int main(int argc, char* argv[])
{
  int i, a, n = 5;
  #pragma omp parallel for private(i) lastprivate(a)
  for (i=0; i<n; i++)
    {
      a = i+1;
      printf("Thread %d has a value of a = %d for i = %d\\n", omp_get_thread_num(),a,i);
    } /*-- End of parallel for --*/
    printf("Value of a after parallel for: a = %d\\n",a);
  }
"""

text_file = open("codeB.c", "w")
text_file.write(code)
text_file.close()
%env OMP_NUM_THREADS=4
!gcc -o Ejecutar -fopenmp codeB.c
!./Ejecutar

env: OMP_NUM_THREADS=4
Thread 0 has a value of a = 1 for i = 0
Thread 0 has a value of a = 2 for i = 1
Thread 2 has a value of a = 4 for i = 3
Thread 3 has a value of a = 5 for i = 4
Thread 1 has a value of a = 3 for i = 2
Value of a after parallel for: a = 5
